In [ ]:
import sys
sys.path.append("../src/")
from utils import LocalPLM, LocalModelArguments

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "microsoft/Phi-4-mini-instruct",
    cuda_devices = "0",
    use_4bit_quantization = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = "float16",
    use_nested_quant = True,
    use_reentrant = True
)

model = LocalPLM(args)

In [ ]:
DATA_PATH = "../discord-chat"

In [ ]:
import glob
def get_chat(user : str, path : str):
    user = "ThisGreenDingo"
    conversation = path + "/Direct Messages - " + user + " [*.csv"
    conversation = glob.glob(conversation)[0]
    

In [ ]:

def read_conversation(path : str) -> pd.DataFrame:
    def parse_time_str(time : str) -> datetime:
        return datetime.strptime(time,'%Y-%m-%dT%H:%M:%S.%f0%z')
        
    c = pd.read_csv(path)
    c["Date"] = c["Date"].map(parse_time_str)
    return c
    
def get_chat(user : str, path : str) -> pd.DataFrame:
    path = path + "/Direct Messages - " + user + " [*.csv"
    conversation = glob.glob(path)

    if conversation:
        return read_conversation(conversation[0])
    else:
        raise FileNotFoundError(f"No conversation(s) found at path {path}")

In [ ]:
c = get_chat("steamingsheep", DATA_PATH)

In [ ]:
c.head()

In [ ]:
def to_string(messages : pd.DataFrame) -> str:
    messages = messages

    end_time = messages.iloc[-1].Date
    
    string = "Conversation history between " + ", ".join(c.Author.unique())
    string += "\n" + end_time.strftime("%y/%m/%d, %H:%M:%S") + "\n\n"
    for i, message in messages.iterrows():
        string += f"{message.Author} {message.Date.strftime("%H:%M:%S")}"
        string += f"\n{message.Content}\n\n"
    
    return string

In [ ]:
def gen_prompt(messages : pd.DataFrame, prompt : str, context : str | None = None) -> str:
    messages = to_string(messages)

    #prompt = "Read the following conversation history and tell me what you think alzter was thinking."
    #prompt = "Read the following conversation history and tell me what you think the relationship is between the users. Answer succinctly."
    #prompt = "Read the following conversation history and tell me what was discussed. Answer succinctly."
    
    prompt = [{"role":"system","content":prompt}]

    if context:prompt.append({"role":"system","content":f"Context: {context}"})
    
    prompt.append({"role":"user","content":messages})

    return prompt

In [ ]:
def understand_conversation(messages : pd.DataFrame, context : str | None = None) -> dict:
    relationship_prompt= "Read the following conversation history and tell me what you think the relationship is between the users. Answer succinctly."
    relationship_prompt = gen_prompt(messages, relationship_prompt, context=context)
    relationship = model.generate(relationship_prompt,temperature=1,max_new_tokens = 128).text
    
    topic_prompt="Read the following conversation history and tell me what was discussed. Answer succinctly."
    topic_prompt = gen_prompt(messages, topic_prompt, context=relationship)
    #print(topic_prompt)
    topic = model.generate(topic_prompt,temperature=1,max_new_tokens = 128).text

    return {"relationship":relationship,"topic":topic}

In [ ]:
understand_conversation(c[:10], "alzter and steamingsheep are brothers.")

In [ ]:
understand_conversation(c[40:52])